# Wormhole + Python - Make your ICO in BCH using Python.

Tutorial by Mau Hernandes (https://twitter.com/mauhcs)

Last edited in 2018/08/22.

In [1]:
import time
import json
import datetime
from requests import post
from collections import defaultdict
from multiprocessing import Process
from IPython.display import clear_output

from cashaddress.convert import to_legacy_address
# If you don't have cashaddress, simple run 'pip install cashaddress' 

# Table of Contens

## 0. Config

## 1. Setup

## 2. Getting Wormhole cash (WHC)

## 3. Getting Addresses Balance

## 4. Issuing Tokens

### 4.1 Fixed Amount 
### 4.2 Managed Amount
### 4.3 Crowd Sale (ICO)

# 5. What is next?

______

# 0. Config
### Set your node rpc endppoint and user:password

In [2]:
RPC_BASEURL="http://178.128.221.48:18332" 
RPC_PASSWORD="h4x0r" 
RPC_USERNAME="l33t" 

## Utility Functions

### Bch and Generic functions

In [3]:
def rpc_call(method, params=[], force_str=True):
    """ Call rpc client method in the remote node.
    List of rpc calls: https://en.bitcoin.it/wiki/Original_Bitcoin_client/API_calls_list
    """
    assert isinstance(params,list), "Parameters MUST be list"
    if force_str:
        _params = [str(p) for p in params]
    else:
        _params = params
    headers = {'content-type': 'text/plain'}
    data = f'{{"jsonrpc": "1.0","id":"{method}" , "method": "{method}", "params": {json.dumps(_params)} }}'
    response = post(RPC_BASEURL, 
                    headers=headers, 
                    data=data,
                    auth=(RPC_USERNAME, RPC_PASSWORD))
    
    ret = response.json()["result"]
    
    if isinstance(ret, str) and len(ret) == 64:
        # Probably a TX.
        return f"https://www.blocktrail.com/tBCC/tx/{ret}"
    else:
        return ret


def get_bch_balances():
    """ Return dict {cashAddress : bch_balance}
    """
    bals = defaultdict(lambda : 0)
    
    txos = rpc_call("listunspent")
    for txo in txos:
        bals[txo["address"]] = round(bals[txo["address"]] + txo["amount"],8)
    return bals

def get_bch_balance(account):
    bal = rpc_call("getbalance", [account])
    return bal

def to_unixtime(year, month, day, hour, minute):
    dt = datetime.datetime(year, month, day, hour, minute)
    return int(time.mktime(dt.timetuple()))


### Token Related Functions

In [4]:
def get_token_id(token, return_one=True):
    """
        Given the name of a Token, returns its ID in Wormhole Network.
        token: String with the name of the token.
        return_one == False return the list of ids matching the name (list).
        return_one == True checks if tehre is only on ID matching the name. If yes, return the id (int).
    """
    _tokens = get_token_names()
    ixs = []
    for ix, name in _tokens.items():
        if token == name:
            ixs.append(ix)
    if len(ixs) == 1 and return_one:
        return ixs[0]
    elif len(ixs) == 0:
        raise TypeError("Unknown Token Name, make sure it was issued")
    elif len(ixs) > 1 and return_one:
        raise TypeError(f"There are multiple Tokens with same name. Ids: {ixs} \n" 
                        "           To avoid this error and return the list of all ids call: \n" 
                        f"           {get_token_id.__name__}({token}, return_one=False)")
    else:
        return ixs

# 1. Setup

## Create 3 Addresses to be used in the tutorial

In [6]:
cashAddresses = {}
for i in range(3):
    addr = rpc_call("getaccountaddress", [str(i)])
    cashAddresses[i] = addr
    print("Account Name:",i , "cashAddress",addr)

Account Name: 0 cashAddress bchtest:qrda8deqt6e0kcrdx2tzvtx0denn0wupvse9t48a8e
Account Name: 1 cashAddress bchtest:qqh7slg0wkk6kt8sftjpt26x7d5sn3v7kqrk3apk3y
Account Name: 2 cashAddress bchtest:qq9zf0cy3k8dpt4x22ydhhwuq8u0hyq9hcx9zyhm03


## Getting tBCHs in your addresses:

### Get a wallet: 

Create a wallet in testnet in this link:

https://ccoin.cash/#wallet

1. Click in settings and change the wallet to testnet.
2. Click in Wallet, enter an email address and a password to create an wallet (Note that the email and password is used only as deterministic noise to generate the wallet uniquely)
3. Copy the (legacy) address under the QRcode shwon in the wallet tab.


### Get tBCH:

Go to this link:
https://testnet.manu.backend.hamburg/bitcoin-cash-faucet

There, paste the (legacy) address to the input field. Fill the Recaptcha and enjoy your tBCH.

### Send Enough to run this tutorial to the new addresses:

Back to the wallet (https://ccoin.cash/#wallet), send tBCHs to your new generated addresses. Since this wallet understands only legacy addresses run the next cell:

In [13]:
# You need the cashaddress for this cell.
# from cashaddress.convert import to_legacy_address
# If you don't simple run 'pip install cashaddress' 

for i in cashAddresses.keys():
    print(f"Address {i}: {to_legacy_address(cashAddresses[i])}")

Address 0: n1ZHuJcN62Jx4E9qK4kcv8GZnZvjcFYWJD
Address 1: mjtGZey4gk1GKneHEpdHtUr2AjLgsNcq5i
Address 2: mgSb596v4dwQbqGEx2R28jbfgiV7hxpgXb


### Send 1.1 tBCHs to address 0
### Send 0.01 tBCHs to address 1
### Send 0.01 tBCHs to address 2

# 2. Getting Wormhole cash (WCH)

Wormhole cash is the money of wormhole layer. Each time you issue a token you must pay WCH. Also you can use it to by tokens in crowd sales (ICOs). The are mainly two ways to get WCHs: 1) Burning BCHs; and 2) Buying in exchange.

Since we are in the testnet we do not need to worry about burning BCHs. As Wormhole whitepaper says, to create WHCs an address should send at least one BCH to the address **bchtest:qqqqqqqqqqqqqqqqqqqqqqqqqqqqqdmwgvnjkt8whc** with some extra information in the op_return. 


To avoid any issue, Wormhole guys were pretty nice to us and came up with an extra rpc call in their Omini Layer implementation **whc_burnbchgetwhc**.

Also, note that when running in the mainnet, you probably should buy WHCs in some exchange instead of burning your BCH. Be Eco Friendly here.

In [12]:
r = rpc_call("whc_burnbchgetwhc", [1, cashAddresses[0]])
print(r)

https://www.blocktrail.com/tBCC/tx/9b65a5e1e597420d2af502ce157c7966eb175e8a57da0351cad20ff7e579c170


Now you just have to wait for your transaction to be mined and you are ready to start issuing your tokens. While you wait, let's track your balance.

# 3. Getting Addresses Balance

In [8]:
# We need To ask the Wormhole network what are the names of all issued Tokens so we can display in our "Wallet"
def get_token_names():
    Ts = rpc_call("whc_listproperties")
    tokens = {}
    for t in Ts:
        tokens[t["propertyid"]] = t["name"]
    return tokens

# Get all the tokens owned by a given address
def get_token_balance(address):
    r = rpc_call("whc_getallbalancesforaddress", [address])
    if r is None:
        return []
    else:
        return r

#### Monitoring balances

The next cell run a parallel process to display the addresses balances in real time.

In [16]:
def loop_balance():
    #tokens = get_token_names()
    tokens = {}
    SLEEP = 10
    while True:
        line = "" 
        bch_bals = get_bch_balances()
        for ix in cashAddresses.keys():
            addr = cashAddresses[ix]

            token_bal = get_token_balance(addr)
            line += f"{ix} : {addr} \n"
            
            #bch = get_bch_balance(str(ix))
            bch = bch_bals[addr]
            line += f" BCH : {bch}" + "\n"
            line += " --- Tokens ---\n"
            
            for b in token_bal:
                try:
                    line += f"{b['propertyid']:>5} {tokens[b['propertyid']]:>7} : {b['balance']}" + "\n"
                except KeyError:
                    if b['propertyid'] not in tokens:
                        # Update tokens:
                        tokens = get_token_names()
        clear_output()
        print("Last Refresh:",datetime.datetime.now())
        print(line, flush=True)
        time.sleep(SLEEP)

# The try block makes sure there is only one loop_balance running in the backgroud.
try:         
    if loop_balance_process.is_alive():
        loop_balance_process.terminate()
except NameError:
    pass

loop_balance_process = Process(target = loop_balance)
loop_balance_process.start()

Last Refresh: 2018-08-22 20:06:41.473458
0 : bchtest:qrda8deqt6e0kcrdx2tzvtx0denn0wupvse9t48a8e 
 BCH : 1.20970369
 --- Tokens ---
1 : bchtest:qqh7slg0wkk6kt8sftjpt26x7d5sn3v7kqrk3apk3y 
 BCH : 0.07
 --- Tokens ---
2 : bchtest:qq9zf0cy3k8dpt4x22ydhhwuq8u0hyq9hcx9zyhm03 
 BCH : 0.07
 --- Tokens ---



Wait for the WHCs tokens to appear and you are ready to continue to issue your first token.

# 4. Issuing Tokens

## Fixed Supply

Let's say you own a Japanese Food Restaurant and want to issue tokens that people can redeem for Temakis.

You decide to issue 100,000 Tokens and will exchange every 10 tokens for one Temaki in your restaurant.

There will not be fractional tokens (precision is zero), and the ticker for the token is **TMK**.

With your idea in mind, you just need to run the next cell and your token will be issued.

In [14]:
from_address      = cashAddresses[0]
ecosystem         = 1 # Must be 1
token_precision   = 0 # Decimal places of the token. Anything between 0 to 8 - [0,8]
previousID        = 0 # 0 if the token is issued for the first time
category          = "Food"
subcategory       = "Japanese"
ticker            = "TMK" # GIVE YOUR UNIQUE NAME TO THIS TOKEN ! (next cells will crash if name conflicts)
url               = "https://en.wikipedia.org/wiki/Sushi"
token_description = "Temaki Token"
total_issued      = 100000 

ret = rpc_call("whc_sendissuancefixed", 
                 [
                     from_address,
                     int(ecosystem),
                     int(token_precision),
                     int(previousID),
                     category,
                     subcategory,
                     ticker,
                     url,
                     token_description,
                     str(total_issued) # Must be string
                 ],
               force_str=False
              )
print(ret)

https://www.blocktrail.com/tBCC/tx/bc5a78a1d2ae21a6be2e11a41755f4a6f0c922a663f25bf676bba04f01e17cc0


### Check if your token was issued:

After issuing the token, you still have to wait for the TX to be mined. 
You can verify your token was mines if the ticker appears in the following dictionary:

In [17]:
# Reminder: The following function calls `whc_listproperties` in the node
get_token_names()


{1: 'WHC',
 3: 'test_token1',
 4: 'test_token1',
 5: 'test_token1',
 6: 'ludete',
 7: 'crowsale_5',
 8: '该',
 9: 'www',
 10: 'snow2',
 11: '3',
 12: '我的',
 13: 'test_managed',
 14: 'zc_fix',
 15: '      bhh',
 16: '精度测试',
 17: 'ludete555',
 18: 'test_bitcoin',
 19: 'qshuai6test',
 20: 'dwdqw',
 21: 'qshuai2fixed',
 22: 'zc_fix2',
 23: 'zc_fix3',
 24: 'zc_fix4',
 25: 'zc_fix5',
 26: 'qshuai1crowsale',
 27: 'zc_crow2',
 28: 'qshuai1crowsale',
 29: 'test_bitcoin',
 30: 'qshuaifinalcrowsale',
 31: 'qshuaifinalcrowsale',
 32: '最新测试',
 33: '小数',
 34: '1.0.6精度测试',
 35: '㊣新虫洞节点测试',
 36: '1.0.7',
 37: '新节点测试',
 38: '1.0.7测试',
 39: '明明是你是男是女是你男生',
 40: '1.0.8测试',
 41: '看大家假的假的几点到家加',
 42: '1.0.1.9测试',
 43: '1.0.1.9测试',
 44: '发发发',
 45: 'test_bitcoin_3',
 46: '1.0.10测试',
 47: 'much',
 48: 'Z',
 49: '1.0.11测试',
 50: '1.0.11测试1',
 51: '1.0.11测试2',
 52: 'xue_token',
 53: 'zc_crowdA',
 54: 'hhj_bitcoin_02',
 55: '推广1',
 56: 'hhj_crowd_01',
 57: 'hhj_crowd_01',
 58: '推广2',
 59: '推广2',
 60: '推广3',
 61:

### Check Token property in the network:

You can check the properties of an issued token, including yours, running the next cell

In [520]:
rpc_call("whc_getproperty", [get_token_id(ticker)], force_str=False)

{'category': 'Food',
 'creationtxid': '1e588f334eb7d2109cb90a336563bcfea242779909aa2dcc8042c967402cddec',
 'data': 'Temaki Token',
 'fixedissuance': True,
 'issuer': 'bchtest:qztl72uj899sh3ztyv9l93pv2uul4nev0u2awg0srr',
 'managedissuance': False,
 'name': 'TMK',
 'precision': 0,
 'propertyid': 167,
 'subcategory': 'Japanese',
 'totaltokens': '100000',
 'url': 'https://en.wikipedia.org/wiki/Sushi'}

**managedissuance** is the flag saying that the token has a fixed amount of tokens. 

**propertyid** is the unique identifier of your token in the wormhole. 

## Sending Tokens

Before we go to other types of token issuance, let's send some tokens to our other addresses:

In [529]:
def send_tokens(fromAddress, toAddress, tokenID, amount):
    """  Sends `amount` tokens with id `tokenID` to address `toAddress`.
    Returns txid of transaction (link to follow in blockexplorer).
    """
    r = rpc_call("whc_send", [fromAddress, toAddress, int(tokenID), str(amount)], force_str=False)
    return r


In [532]:
# Send 10 Tokens to cashAddress 1
r = send_tokens(cashAddresses[0], cashAddresses[1], get_token_id(ticker), 10)
print(r)

'https://www.blocktrail.com/tBCC/tx/90c65d355a9b9e133b56408b549fada013763b8b217fdf23b9dd955f44bd942e'

In [530]:
# Send 20 Tokens to cashAddress 2
r = send_tokens(cashAddresses[0], cashAddresses[2], get_token_id(ticker), 20)
print(r)

'https://www.blocktrail.com/tBCC/tx/ce1b263ccdf078d9d8e6676a26ef955e36a0becc456df5b7c6bee589741e64d4'

In [533]:
# Uncomment the next line to send me 1 of your Tokens:
#print(send_tokens(cashAddresses[0],"bchtest:qzmcwgjg0drn3vcehuqdwkvpav484k7f8uugqd6slv",get_token_id(ticker),1))

'https://www.blocktrail.com/tBCC/tx/c37b97ee24838f70ad5ae835e42dc1ce714e06c2534bb0efff1ddb8bd1d9a933'

In [600]:
# Since we are sending tokens, let's send some WHC to address 1 and 2 for later in the Tutorial
# Send 10 WHC to cashAddress 1 and 2:
r = send_tokens(cashAddresses[0], cashAddresses[1], 1, 10) # WHC token id is 1
print(r)
r = send_tokens(cashAddresses[0], cashAddresses[1], 1, 10) # WHC token id is 1
print(r)

https://www.blocktrail.com/tBCC/tx/bb0ba445f741da69c484a15ec14d7e8a0aab4c970ca0f69b22054fc977bd2fbb
https://www.blocktrail.com/tBCC/tx/44706654e04543a798780573d9ad74e0500e5680463712ddf592ba4c09fe90e5


## Managed Supply

The Temaki Token was a success.
But now, you want to issue tokens to all sorts of sushis in your Japanese restaurant. Since you cannot predict all types of sushi you will ever have in your menu, you decide to issue tokens that can have a managed supply. Enter the **Managed Supply** tokens.

You decide to call you token **SUSHI** token, and at the time of creation you do not have to decide how many you want. You simply create running the following cell.

In [539]:
from_address      = cashAddresses[0]
ecosystem         = 1 # Must be 1
token_precision   = 0 # Decimal places of the token. Anything between 0 to 8 - [0,8]
previousID        = 0 # 0 if the token is new
category          = "Food"
subcategory       = "Japanese"
ticker            = "SUSHI" # GIVE YOUR UNIQUE NAME TO THIS TOKEN ! (next cells will crash if name conflicts)
url               = "https://en.wikipedia.org/wiki/Sushi"
token_description = "Sushi Token"


ret = rpc_call("whc_sendissuancemanaged", 
                 [
                     from_address,
                     int(ecosystem),
                     int(token_precision),
                     int(previousID),
                     category,
                     subcategory,
                     ticker,
                     url,
                     token_description
                 ],
               force_str=False
              )
print(ret)

https://www.blocktrail.com/tBCC/tx/901d06c9492197c58997dc9185933a784c6db3d93b6d9fe574b0894537e76104


In [580]:
#### Check if the token was issued
# Reminder: The following function calls `whc_listproperties` in the node
get_token_names()


{1: 'WHC',
 3: 'test_token1',
 4: 'test_token1',
 5: 'test_token1',
 6: 'ludete',
 7: 'crowsale_5',
 8: '该',
 9: 'www',
 10: 'snow2',
 11: '3',
 12: '我的',
 13: 'test_managed',
 14: 'zc_fix',
 15: '      bhh',
 16: '精度测试',
 17: 'ludete555',
 18: 'test_bitcoin',
 19: 'qshuai6test',
 20: 'dwdqw',
 21: 'qshuai2fixed',
 22: 'zc_fix2',
 23: 'zc_fix3',
 24: 'zc_fix4',
 25: 'zc_fix5',
 26: 'qshuai1crowsale',
 27: 'zc_crow2',
 28: 'qshuai1crowsale',
 29: 'test_bitcoin',
 30: 'qshuaifinalcrowsale',
 31: 'qshuaifinalcrowsale',
 32: '最新测试',
 33: '小数',
 34: '1.0.6精度测试',
 35: '㊣新虫洞节点测试',
 36: '1.0.7',
 37: '新节点测试',
 38: '1.0.7测试',
 39: '明明是你是男是女是你男生',
 40: '1.0.8测试',
 41: '看大家假的假的几点到家加',
 42: '1.0.1.9测试',
 43: '1.0.1.9测试',
 44: '发发发',
 45: 'test_bitcoin_3',
 46: '1.0.10测试',
 47: 'much',
 48: 'Z',
 49: '1.0.11测试',
 50: '1.0.11测试1',
 51: '1.0.11测试2',
 52: 'xue_token',
 53: 'zc_crowdA',
 54: 'hhj_bitcoin_02',
 55: '推广1',
 56: 'hhj_crowd_01',
 57: 'hhj_crowd_01',
 58: '推广2',
 59: '推广2',
 60: '推广3',
 61:

In [549]:
# Check the Token Properties
rpc_call("whc_getproperty", [get_token_id(ticker)], force_str=False)

{'category': 'Food',
 'creationtxid': '901d06c9492197c58997dc9185933a784c6db3d93b6d9fe574b0894537e76104',
 'data': 'Sushi Token',
 'fixedissuance': False,
 'freezingenabled': False,
 'issuer': 'bchtest:qztl72uj899sh3ztyv9l93pv2uul4nev0u2awg0srr',
 'managedissuance': True,
 'name': 'SUSHI',
 'precision': 0,
 'propertyid': 168,
 'subcategory': 'Japanese',
 'totaltokens': '0',
 'url': 'https://en.wikipedia.org/wiki/Sushi'}

### Issuing the Managed Tokens
Later on, when you have costumers coming to your restaurant you issue tokens each time they pay using BCH. 
One thousand SUSHI tokens per client, you are eager to have more payments coming in BCH, and you want to see the client come back.

In [620]:
def issue_tokens(fromAddress, toAddress, tokenID, amount, memo=""):
    r = rpc_call(method = "whc_sendgrant", 
                 params = [fromAddress, toAddress, int(tokenID), str(amount), str(memo)], 
                 force_str=False)
    return r

In [557]:
r = issue_tokens(cashAddresses[0], cashAddresses[1], get_token_id(ticker), 1000, "Sushi AirDrop")
print(r)

'https://www.blocktrail.com/tBCC/tx/4cf135e9130cd0f7ac3d7854cb303b7e53bbbf2e5c5845f95510bc1da1df4560'

In [558]:
r = issue_tokens(cashAddresses[0], cashAddresses[2], get_token_id(ticker), 1000, "Sushi AirDrop")
print(r)

'https://www.blocktrail.com/tBCC/tx/a4cfd2b9da29e19b1e5de320c1335ef2b5845954984c56b5c9c81f039275c07d'

In [560]:
# Uncomment the next line to send me 1 of your Tokens:
#print(issue_tokens(cashAddresses[0],"bchtest:qzmcwgjg0drn3vcehuqdwkvpav484k7f8uugqd6slv",get_token_id(ticker),1))

### Changing Issuer - Selling the business.

Your business was very sucessful, part because you started accepting BCH in your restaurant. As you wanted to explorer other possibilities you decided to sell the bussines. With it, you need to transfer the issuer of your SUSHI tokens to the new owner.

For that, you just have to run the next few cells.

In [563]:
def change_issuer(fromAddress, toAddress, tokenID):
    r = rpc_call(method = "whc_sendchangeissuer",
                 params = [fromAddress, toAddress, int(tokenID)],
                 force_str=False)
    return r                 

In [564]:
r = change_issuer(cashAddresses[0], cashAddresses[1], get_token_id(ticker))
print(r)

'https://www.blocktrail.com/tBCC/tx/8f20bc799401aa8e36786685b0e38833e65192cf5122646c33304148629f0be7'

In [567]:
# Check the Token Properties
rpc_call("whc_getproperty", [get_token_id(ticker)], force_str=False)

{'category': 'Food',
 'creationtxid': '901d06c9492197c58997dc9185933a784c6db3d93b6d9fe574b0894537e76104',
 'data': 'Sushi Token',
 'fixedissuance': False,
 'freezingenabled': False,
 'issuer': 'bchtest:qr6jk8rfrssp6z5d60w4943e640ktex3vydum7njhx',
 'managedissuance': True,
 'name': 'SUSHI',
 'precision': 0,
 'propertyid': 168,
 'subcategory': 'Japanese',
 'totaltokens': '4001',
 'url': 'https://en.wikipedia.org/wiki/Sushi'}

____

The new owner can keep the bussiness of issuing SUSHI tokens without any problem, and you follow your dreams as you plan an ICO to disrupt the Tuna market.

In [568]:
r = issue_tokens(cashAddresses[1], cashAddresses[2], get_token_id(ticker), 1000, "Sushi AirDrop")
print(r)

'https://www.blocktrail.com/tBCC/tx/ac7e258eff5943a86f9b2824395dca88a6b1f18b8265ce9a50492b9bc9158e76'

## Crowd Sale (ICO)

You became an expert in crypto fishes in Japan. You see the demand for Tuna growing around the world, and to seize the opportunity you decide to run an ICO of **TUNA** tokens.

- You decide to issue 1,000,000 tokens.

- You will accept WHC as payment. 

- 1 WHC will buy 20 TUNA tokens

- No Decimal tokens.

- You will hold the ICO for a month (Starting August 22, ending September 22).

- 30% Early Bonus per week prior to the deadline.

- The Token will be called **TUNA**.

With all that decided, you just have to run the next cell and your ICO will be up and running:

In [587]:
from_address      = cashAddresses[0]
ecosystem         = 1 # Must be 1
token_precision   = 0 # Decimal places of the token. Anything between 0 to 8 - [0,8]
previousID        = 0 # 0 if the token is new
category          = "Food"
subcategory       = "Japanese"
ticker            = "TUNA" # GIVE YOUR UNIQUE NAME TO THIS TOKEN ! (next cells will crash if name conflicts)
url               = "https://en.wikipedia.org/wiki/Sushi"
token_description = "Maguro(Tuna) Token"
tokenID_desired   = 1 # 1 for WHC 
token_per_unit    = 20
deadline          = to_unixtime(2018, 9, 22, 0, 0)
early_bonus       = 30
undefine          = 0 # Must be zero
total_supply      = 1000000

ret = rpc_call("whc_sendissuancecrowdsale", 
                 [
                     from_address,
                     int(ecosystem),
                     int(token_precision),
                     int(previousID),
                     category,
                     subcategory,
                     ticker,
                     url,
                     token_description,
                     int(tokenID_desired),
                     str(token_per_unit),
                     int(deadline),
                     int(early_bonus),
                     undefine,
                     str(total_supply)
                 ],
               force_str=False
              )
print(ret)

https://www.blocktrail.com/tBCC/tx/48d8c3bacf8e0b6e332604d35bc73409b959a70ae5d39b4ef6e1d45d661ef66f


In [592]:
#### Check if the token was issued
# Reminder: The following function calls `whc_listproperties` in the node
get_token_names()


{1: 'WHC',
 3: 'test_token1',
 4: 'test_token1',
 5: 'test_token1',
 6: 'ludete',
 7: 'crowsale_5',
 8: '该',
 9: 'www',
 10: 'snow2',
 11: '3',
 12: '我的',
 13: 'test_managed',
 14: 'zc_fix',
 15: '      bhh',
 16: '精度测试',
 17: 'ludete555',
 18: 'test_bitcoin',
 19: 'qshuai6test',
 20: 'dwdqw',
 21: 'qshuai2fixed',
 22: 'zc_fix2',
 23: 'zc_fix3',
 24: 'zc_fix4',
 25: 'zc_fix5',
 26: 'qshuai1crowsale',
 27: 'zc_crow2',
 28: 'qshuai1crowsale',
 29: 'test_bitcoin',
 30: 'qshuaifinalcrowsale',
 31: 'qshuaifinalcrowsale',
 32: '最新测试',
 33: '小数',
 34: '1.0.6精度测试',
 35: '㊣新虫洞节点测试',
 36: '1.0.7',
 37: '新节点测试',
 38: '1.0.7测试',
 39: '明明是你是男是女是你男生',
 40: '1.0.8测试',
 41: '看大家假的假的几点到家加',
 42: '1.0.1.9测试',
 43: '1.0.1.9测试',
 44: '发发发',
 45: 'test_bitcoin_3',
 46: '1.0.10测试',
 47: 'much',
 48: 'Z',
 49: '1.0.11测试',
 50: '1.0.11测试1',
 51: '1.0.11测试2',
 52: 'xue_token',
 53: 'zc_crowdA',
 54: 'hhj_bitcoin_02',
 55: '推广1',
 56: 'hhj_crowd_01',
 57: 'hhj_crowd_01',
 58: '推广2',
 59: '推广2',
 60: '推广3',
 61:

In [597]:
rpc_call("whc_getcrowdsale", 
        params = [get_token_id(ticker), True],
        force_str = False)

{'active': True,
 'addedissuertokens': '0',
 'amountraised': '0.00000000',
 'deadline': 1537542000,
 'earlybonus': 30,
 'issuer': 'bchtest:qztl72uj899sh3ztyv9l93pv2uul4nev0u2awg0srr',
 'name': 'TUNA',
 'participanttransactions': [],
 'precision': '0',
 'propertyid': 169,
 'propertyiddesired': 1,
 'starttime': 1534926695,
 'tokensissued': '1000000',
 'tokensperunit': '20.00000000'}

### Buy tokens in the ICO
Now, anyone willing to buy your Tokens can do that directly by sending a transaction to the bitcoin cash network. For that, just run the next few cells:

In [598]:
def buy_token(buyer_address, seller_address, amount):
    r = rpc_call("whc_particrowsale", [buyer_address, seller_address, amount])
    return r

In [601]:
r = buy_token(buyer_address=cashAddresses[1], seller_address=cashAddresses[0], amount=0.5)
print(r)

'https://www.blocktrail.com/tBCC/tx/a471ae1e2c8b9e7110b16470983666b3cf07af748fd8030137cf0b428d53e41d'

You can monitor your ICO through the blockchain itself as the record of each buyer is kept. To recover this information just run the next cell and look for `participanttransactions`.

In [603]:
rpc_call("whc_getcrowdsale", 
        params = [get_token_id(ticker), True],
        force_str = False)

{'active': True,
 'addedissuertokens': '0',
 'amountraised': '0.47901202',
 'deadline': 1537542000,
 'earlybonus': 30,
 'issuer': 'bchtest:qztl72uj899sh3ztyv9l93pv2uul4nev0u2awg0srr',
 'name': 'TUNA',
 'participanttransactions': [{'amountsent': '0.47901202',
   'participanttokens': '22',
   'txid': 'a471ae1e2c8b9e7110b16470983666b3cf07af748fd8030137cf0b428d53e41d'}],
 'precision': '0',
 'propertyid': 169,
 'propertyiddesired': 1,
 'starttime': 1534926695,
 'tokensissued': '1000000',
 'tokensperunit': '20.00000000'}

As you can see our your first ICO participant sent 0.479 WCH to your address and received 22 TUNA tokens. The reason is due to Early bonus of 30% per week, and there are 4 weeks to the end of the sale. 

In [649]:
# How the 22 
seconds_in_seven_days = 7 * 24 * 60 * 60

buytime = 1534928471 # Given by the blocktime in which the TX was mined
deadline = 1537542000 # Given by whc_getcrowdsale rpc call

weeks_left = round((deadline - buytime) / seconds_in_seven_days,8)
print("Weeks Left:", weeks_left)
bonus = 1 + (0.3 * weeks_left ) # 30% times 4.32 (Number of weeks until the end of the sale)
print("Bonus", bonus)
print("Tokens", round(0.47901202 * 20 * bonus, 8))

Weeks Left: 4.32131118
Bonus 2.296393354
Tokens 22.00000038


# 5. What next?

Wormhole's whitepaper has a strong and long roadmap, including wallets and smart-contracs. Another aspect the team should be working on the next months is its documentation. For now, the best references to read aboud wormholed documentation is ominicore rpc github webpage:
    https://github.com/OmniLayer/omnicore/blob/master/src/omnicore/doc/rpc-api.md

Since Wormhole is a fork of omini layer, most of the *omini_method-name* in the docs are ported as whc_method-name. This means that, if you want to know an example of how to call some particular rpc function that I did not explain (For example, what is omini_getsto ?), get yourself a wormholed and run:

        wormholed-cli whc_getsto
This will prompt an error message explaining how to use this function. 

Other really good source of reference for Wormhole is https://twitter.com/cgcardona. He put together a three part tutorial on wormhole and some NodeJs libraries to run your own Wormhole project. 

pt 1 https://developer.bitcoin.com/tutorials/wormhole-1-setup.html

pt 2 https://developer.bitcoin.com/tutorials/wormhole-1-setup.html
    
pt 3 https://developer.bitcoin.com/tutorials/wormhole-3-tokens.html

This Jupyter Notebook Tutorial here is basically pt 3 reworked for python. Good stuff, but if you want to learn how to run your own full node, check pt 2 as well in this series of tutorials.

Finally, from the same developer, this NodeJs application of a restApi is a good source for reference on Wormhole. 
Specially the pages for data retrieval and transactions:

https://github.com/Bitcoin-com/wormholerest.bitcoin.com/blob/master/routes/dataRetrieval.js

https://github.com/Bitcoin-com/wormholerest.bitcoin.com/blob/master/routes/transaction.js